In [1]:
!pip install numpy pandas scikit-learn imbalanced-learn xgboost shap lime matplotlib seaborn joblib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=f2b318ef12891eac62f3a4a91d2ee6ca17c4acce5d9ff402d9061ea1da402eea
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, recall_score
from imblearn.under_sampling import RandomUnderSampler
import xgboost as xgb
import shap
import lime
from lime import lime_tabular
from scipy.stats import spearmanr
from sklearn.inspection import permutation_importance
import joblib
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style for publication-ready plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

# Create project directory structure
directories = ['data', 'models', 'results', 'plots', 'xai', 'preprocessing']
for dir_path in directories:
    os.makedirs(dir_path, exist_ok=True)

print(" XAI Imbalanced Learning Pipeline Started (COMPLETE WITH PLOTS)")


class XAIImbalanceStudy:
    def __init__(self):
        self.scaler = StandardScaler()
        self.results = {}
        self.feature_names = None

    def load_creditcard_data(self):
        """Load and preprocess Credit Card Fraud dataset"""
        try:
            df = pd.read_csv('/content/creditcard.csv')
            print(f" Dataset loaded: {df.shape}")
            print(f"   Fraud rate: {df['Class'].mean():.4f}")
        except FileNotFoundError:
            print(" Dataset not found! Download from:")
            print("   https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud")
            print("   Place 'creditcard.csv' in 'data/' folder")
            return None

        # Preprocessing: Drop Time, handle any issues
        df = df.drop('Time', axis=1)
        self.feature_names = df.drop('Class', axis=1).columns.tolist()

        # Save preprocessed data
        df.to_csv('preprocessing/creditcard_preprocessed.csv', index=False)
        return df

    def create_imbalance_levels(self, X, y, test_size=0.2, random_state=42):
        """Create systematic imbalance ratios: 1:1, 1:5, 1:10, 1:50, 1:100"""
        print("\n Creating imbalance levels...")

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=y
        )

        imbalance_ratios = {
            'balanced_1_1': 1,
            'mild_1_5': 5,
            'moderate_1_10': 10,
            'severe_1_50': 50,
            'extreme_1_100': 100
        }

        train_data = {}
        for ratio_name, target_ratio in imbalance_ratios.items():
            n_minority = sum(y_train == 1)
            n_majority_target = min(n_minority * target_ratio, sum(y_train == 0))

            rus = RandomUnderSampler(
                sampling_strategy={0: n_majority_target},
                random_state=random_state
            )
            X_res, y_res = rus.fit_resample(X_train, y_train)

            train_data[ratio_name] = (X_res, y_res)
            print(f"   {ratio_name}: {Counter(y_res)} (ratio {target_ratio}:1)")

        return train_data, (X_test, y_test)

    def train_models(self, X_train, y_train):
        """Train all models with FIXED hyperparameters for fair comparison"""
        models = {
            'LogisticRegression': LogisticRegression(
                random_state=42, max_iter=2000, C=1.0
            ),
            'RandomForest': RandomForestClassifier(
                n_estimators=100, max_depth=10, random_state=42, n_jobs=-1
            ),
            'XGBoost': xgb.XGBClassifier(
                n_estimators=100, max_depth=5, learning_rate=0.1,
                random_state=42, n_jobs=-1, eval_metric='logloss'
            )
        }

        trained_models = {}

        for name, model in models.items():
            print(f"      Training {name}...", end=" ")

            if 'LogisticRegression' in name:
                X_train_scaled = self.scaler.fit_transform(X_train)
                model.fit(X_train_scaled, y_train)
                trained_models[name] = {'model': model, 'scaler': self.scaler}
            else:
                model.fit(X_train, y_train)
                trained_models[name] = {'model': model, 'scaler': None}



        return trained_models

    def evaluate_performance(self, models, X_test, y_test):
        """Compute standard imbalanced classification metrics"""
        perf_results = {}

        for name, model_info in models.items():
            model = model_info['model']
            scaler = model_info['scaler']

            if scaler is not None:
                X_test_scaled = scaler.transform(X_test)
                y_pred = model.predict(X_test_scaled)
                y_proba = model.predict_proba(X_test_scaled)[:, 1]
            else:
                y_pred = model.predict(X_test)
                y_proba = model.predict_proba(X_test)[:, 1]

            perf_results[name] = {
                'F1_score': f1_score(y_test, y_pred),
                'PR_AUC': average_precision_score(y_test, y_proba),
                'ROC_AUC': roc_auc_score(y_test, y_proba),
                'Recall_minority': recall_score(y_test, y_pred)
            }
        return perf_results

    def explanation_stability_score(self, importance_lists):
        """ESS: Spearman correlation of feature rankings across multiple runs"""
        if len(importance_lists) < 2:
            return 0.0

        rankings = []
        for importance in importance_lists:
            if len(importance) == 0:
                rankings.append(np.zeros(1))
                continue
            # Get feature ranking by absolute importance
            ranking = np.argsort(np.abs(importance))[::-1]
            rankings.append(ranking)

        # Compute all pairwise Spearman correlations
        correlations = []
        for i in range(len(rankings)):
            for j in range(i+1, len(rankings)):
                if len(rankings[i]) == len(rankings[j]):
                    corr, _ = spearmanr(rankings[i], rankings[j])
                    correlations.append(corr)

        return np.mean(correlations) if correlations else 0.0

    def feature_importance_drift(self, importance_lists):
        """FIXED FID: Jaccard distance of top-10 features across runs"""
        if len(importance_lists) < 2:
            return 0.0

        drifts = []
        top_k = 10

        for i in range(len(importance_lists)):
            for j in range(i+1, len(importance_lists)):
                importance_i = np.abs(importance_lists[i])
                importance_j = np.abs(importance_lists[j])

                # Get top-K feature INDICES (integers) - FIX FOR TypeError
                top_features_i = set(np.argsort(importance_i)[-top_k:].tolist())
                top_features_j = set(np.argsort(importance_j)[-top_k:].tolist())

                # Jaccard similarity using integer indices
                intersection = len(top_features_i.intersection(top_features_j))
                union = len(top_features_i.union(top_features_j))
                jaccard_sim = intersection / union if union > 0 else 0
                drifts.append(1 - jaccard_sim)  # Distance = 1 - similarity

        return np.mean(drifts) if drifts else 0.0

    def get_model_predictions(self, model_info, X):
        """Helper to get predictions with proper scaling"""
        model = model_info['model']
        scaler = model_info['scaler']
        if scaler is not None:
            X_scaled = scaler.transform(X)
            return model.predict_proba(X_scaled)[:, 1]
        return model.predict_proba(X)[:, 1]

    def generate_shap_explanations(self, model_info, X_train, X_test, n_samples=50):
        """Generate SHAP feature importances - FIXED"""
        try:
            model = model_info['model']
            scaler = model_info['scaler']

            if scaler is not None:
                X_train_scaled = scaler.transform(X_train)
                X_test_scaled = scaler.transform(X_test)
                train_data = X_train_scaled
                test_data = X_test_scaled[:n_samples]
            else:
                train_data = X_train
                test_data = X_test[:n_samples]

            # SHAP explainer based on model type
            if 'XGB' in str(type(model)).upper():
                explainer = shap.TreeExplainer(model)
            elif 'LogisticRegression' in str(type(model)):
                explainer = shap.LinearExplainer(model, train_data)
            else:
                explainer = shap.KernelExplainer(
                    lambda x: model.predict_proba(scaler.transform(x) if scaler else x)[:, 1],
                    train_data[:100]
                )

            shap_values = explainer.shap_values(test_data)

            # Handle multi-class output
            if isinstance(shap_values, list):
                shap_values = shap_values[1]  # Positive class

            return np.abs(shap_values).mean(axis=0)
        except Exception as e:
            print(f"SHAP error: {e}")
            return np.zeros(len(self.feature_names))

    def generate_lime_explanations(self, model_info, X_train, X_test, n_samples=30):
        """Generate LIME feature importances - FIXED"""
        try:
            model = model_info['model']
            scaler = model_info['scaler']

            if scaler is not None:
                X_train_scaled = scaler.transform(X_train)
                training_data = X_train_scaled
            else:
                training_data = X_train

            explainer = lime.tabular.LimeTabularExplainer(
                training_data.astype(np.float32),
                feature_names=self.feature_names,
                class_names=['Normal', 'Fraud'],
                mode='classification'
            )

            lime_importances = []
            predict_fn = lambda x: model_info['model'].predict_proba(
                scaler.transform(x) if scaler else x
            )[:, 1]

            for i in range(min(n_samples, len(X_test))):
                try:
                    exp = explainer.explain_instance(
                        X_test[i].astype(np.float32),
                        predict_fn,
                        num_features=len(self.feature_names)
                    )

                    importance_dict = dict(exp.as_list())
                    imp_array = np.zeros(len(self.feature_names))

                    for feat_idx, val in importance_dict.items():
                        imp_array[int(feat_idx)] = abs(val)

                    lime_importances.append(imp_array)
                except:
                    continue

            return np.mean(lime_importances, axis=0) if lime_importances else np.zeros(len(self.feature_names))
        except:
            return np.zeros(len(self.feature_names))

    def run_stability_analysis(self, models, X_train, X_test, ratio_name):
        """Run stability analysis for explanation consistency - FIXED"""
        print(f" Generating explanations (5 runs)")

        stability_results = {}
        n_stability_runs = 5

        for model_name, model_info in models.items():
            shap_importances = []
            lime_importances = []

            print(f"     Processing {model_name}...")

            # Multiple runs for stability assessment
            for run in range(n_stability_runs):
                # SHAP explanations
                shap_vals = self.generate_shap_explanations(model_info, X_train, X_test)
                shap_importances.append(shap_vals)

                # LIME explanations
                lime_vals = self.generate_lime_explanations(model_info, X_train, X_test)
                lime_importances.append(lime_vals)

            # Compute stability metrics
            stability_results[model_name] = {
                'SHAP_ESS': self.explanation_stability_score(shap_importances),
                'SHAP_FID': self.feature_importance_drift(shap_importances),
                'LIME_ESS': self.explanation_stability_score(lime_importances),
                'LIME_FID': self.feature_importance_drift(lime_importances)
            }

        return stability_results

    # ==================== PLOTTING FUNCTIONS ====================
    # ==================== FIXED PLOTTING FUNCTIONS ====================
    def plot_performance_heatmap(self, results):
        """Plot performance metrics heatmap across imbalance ratios - FIXED"""
        print("\n📈 Creating Performance Heatmap...")

        # Extract performance data SAFELY
        perf_data = []
        for ratio in results.keys():
            perf_dict = results[ratio].get('performance', {})
            for model in perf_dict:
                row = {
                    'Ratio': ratio.replace('_', ' ').title(),
                    'Model': model
                }
                # Safely get all metrics
                metrics = perf_dict[model]
                for metric in ['F1_score', 'PR_AUC', 'ROC_AUC', 'Recall_minority']:
                    row[metric] = metrics.get(metric, 0.0)
                perf_data.append(row)

        if not perf_data:
            print("    No performance data available for plotting")
            return

        df_perf = pd.DataFrame(perf_data)
        print(f"   Heatmap data shape: {df_perf.shape}")

        # Melt for heatmap
        metrics = ['F1_score', 'PR_AUC', 'ROC_AUC', 'Recall_minority']
        df_melt = df_perf.melt(id_vars=['Ratio', 'Model'], value_vars=metrics,
                              var_name='Metric', value_name='Score')

        plt.figure(figsize=(14, 10))
        # Create multi-level pivot table
        pivot_table = df_melt.pivot_table(
            values='Score',
            index='Model',
            columns=['Ratio', 'Metric'],
            aggfunc='mean'
        )

        sns.heatmap(pivot_table, annot=True, cmap='RdYlBu_r', fmt='.3f',
                    cbar_kws={'label': 'Performance Score'}, center=0.5)
        plt.title('Model Performance Across Imbalance Ratios\n(F1, PR-AUC, ROC-AUC, Recall)',
                  fontsize=14, pad=20)
        plt.xlabel('Imbalance Ratio × Metric', fontsize=12)
        plt.ylabel('Model', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('plots/performance_heatmap.png', bbox_inches='tight', dpi=300, facecolor='white')
        plt.close()
        print("  Saved: plots/performance_heatmap.png")

    def plot_stability_curves(self, results):
        """Plot stability metrics vs imbalance severity - FIXED"""
        print(" Creating Stability Curves...")

        ratios_order = ['balanced_1_1', 'mild_1_5', 'moderate_1_10',
                       'severe_1_50', 'extreme_1_100']
        x_pos = np.arange(len(ratios_order))
        models = ['LogisticRegression', 'RandomForest', 'XGBoost']
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

        # Collect stability data safely
        stability_data = {model: [] for model in models}

        for ratio in ratios_order:
            if ratio in results and 'stability' in results[ratio]:
                stab_dict = results[ratio]['stability']
                for model in models:
                    ess_score = stab_dict.get(model, {}).get('SHAP_ESS', 0)
                    stability_data[model].append(ess_score)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        fig.suptitle('XAI Explanation Stability vs Imbalance Severity', fontsize=16)

        # ESS Plot (Higher = Better)
        for i, model in enumerate(models):
            scores = stability_data[model]
            if any(s > 0 for s in scores):  # Only plot if we have data
                ax1.plot(x_pos, scores, 'o-', color=colors[i], linewidth=3,
                        label=model, markersize=8, markerfacecolor='white', markeredgewidth=2)

        ax1.set_xticks(x_pos)
        ax1.set_xticklabels([r.replace('_', '\n') for r in ratios_order], fontsize=11)
        ax1.set_ylabel('SHAP ESS (Spearman Correlation)', fontsize=12)
        ax1.set_title('Explanation Stability Score\n(Higher = More Stable)', fontsize=12)
        ax1.legend(frameon=True, fancybox=True, shadow=True)
        ax1.grid(True, alpha=0.3)
        ax1.set_ylim(-0.1, 1.1)

        # Sample sizes
        sample_sizes = [results[r].get('n_samples', 0) for r in ratios_order]
        ax2.bar(x_pos, sample_sizes, color='lightcoral', alpha=0.7, edgecolor='darkred')
        ax2.set_xticks(x_pos)
        ax2.set_xticklabels([r.replace('_', '\n') for r in ratios_order], fontsize=11)
        ax2.set_ylabel('Training Samples', fontsize=12)
        ax2.set_title('Training Set Size per Ratio', fontsize=12)
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig('plots/stability_curves.png', bbox_inches='tight', dpi=300, facecolor='white')
        plt.close()
        print("    Saved: plots/stability_curves.png")

    def plot_model_radar(self, results):
        """Create radar chart comparing models across ratios - NEW"""
        print(" Creating Model Comparison Radar...")

        metrics = ['F1_score', 'PR_AUC', 'ROC_AUC']
        models = ['LogisticRegression', 'RandomForest', 'XGBoost']
        n_metrics = len(metrics)

        # Average performance across ratios
        avg_perf = {}
        for model in models:
            scores = []
            for ratio in results:
                perf = results[ratio].get('performance', {}).get(model, {})
                model_scores = [perf.get(m, 0) for m in metrics]
                scores.append(model_scores)
            avg_perf[model] = np.mean(scores, axis=0)

        angles = np.linspace(0, 2*np.pi, n_metrics, endpoint=False).tolist()
        angles += angles[:1]  # Complete the circle

        fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

        for i, model in enumerate(models):
            values = avg_perf[model].tolist() + avg_perf[model][:1].tolist()
            ax.plot(angles, values, 'o-', linewidth=3, label=model,
                    markersize=8, markerfacecolor='white')
            ax.fill(angles, values, alpha=0.25)

        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(metrics, fontsize=12)
        ax.set_ylim(0, 1)
        ax.set_title('Model Performance Comparison (Avg Across Ratios)\nRadar Chart',
                     fontsize=16, pad=20)
        ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
        ax.grid(True)

        plt.tight_layout()
        plt.savefig('plots/model_radar_comparison.png', bbox_inches='tight', dpi=300, facecolor='white')
        plt.close()
        print("  Saved: plots/model_radar_comparison.png")

    def create_all_plots(self, results):
        """Generate all publication-ready plots - FIXED"""
        print("\n Generating Publication-Ready Plots...")

        try:
            self.plot_performance_heatmap(results)
            self.plot_stability_curves(results)
            self.plot_model_radar(results)
            print("   All plots generated successfully!")
        except Exception as e:
            print(f"   Plotting error: {e}")
            print("    Basic plots still available in results/")

    def create_summary_table(self, results):
        """Create publication-ready summary table"""
        ratios = list(results.keys())
        models = ['LogisticRegression', 'RandomForest', 'XGBoost']
        summary_data = []

        for ratio in ratios:
            for model in models:
                if model in results[ratio]['performance']:
                    row = {'Ratio': ratio, 'Model': model}
                    row.update(results[ratio]['performance'][model])

                    if model in results[ratio]['stability']:
                        row.update({
                            'SHAP_ESS': results[ratio]['stability'][model].get('SHAP_ESS', 0),
                            'LIME_ESS': results[ratio]['stability'][model].get('LIME_ESS', 0)
                        })
                    summary_data.append(row)

        return pd.DataFrame(summary_data)

    def run_complete_experiment(self):
        """Execute FULL experiment pipeline - COMPLETE WITH PLOTS"""
        print("\n Step 1: Loading and preprocessing data")
        df = self.load_creditcard_data()
        if df is None:
            return None

        X = df.drop('Class', axis=1).values
        y = df['Class'].values

        print("\n Step 2: Creating imbalance levels...")
        train_data, (X_test, y_test) = self.create_imbalance_levels(X, y)

        print("\n Step 3: Running experiments across all ratios")
        all_results = {}

        for ratio_name, (X_train, y_train) in train_data.items():
            print(f"\n Processing {ratio_name.upper()}...")

            # Train models
            models = self.train_models(X_train, y_train)

            # Save models
            for name, model_info in models.items():
                joblib.dump(model_info, f'models/{name}_{ratio_name}.pkl')

            # Performance evaluation
            perf_results = self.evaluate_performance(models, X_test, y_test)

            # Stability analysis
            stability_results = self.run_stability_analysis(models, X_train, X_test, ratio_name)

            # Store results
            all_results[ratio_name] = {
                'performance': perf_results,
                'stability': stability_results,
                'n_samples': len(X_train)
            }

            print(f" {ratio_name} completed!")

        # Save comprehensive results
        self.results = all_results
        joblib.dump(all_results, 'results/complete_experiment_results.pkl')

        # Create summary CSV
        summary_df = self.create_summary_table(all_results)
        summary_df.to_csv('results/summary_metrics.csv', index=True)

        # GENERATE ALL PLOTS
        self.create_all_plots(all_results)

        print("\n COMPLETE EXPERIMENT FINISHED WITH PLOTS!")
        print("Check folders: results/, models/, plots/")
        return all_results

# MAIN EXECUTION
if __name__ == "__main__":
    study = XAIImbalanceStudy()
    results = study.run_complete_experiment()

    if results:
        print("\n EXPERIMENT SUCCESSFUL! Generated:")
        print("    plots/performance_heatmap.png")
        print("    plots/stability_curves.png")
        print("    plots/feature_ranking_extreme_1_100.png")
        print("    results/summary_metrics.csv")

 XAI Imbalanced Learning Pipeline Started (COMPLETE WITH PLOTS)

 Step 1: Loading and preprocessing data
 Dataset loaded: (284807, 31)
   Fraud rate: 0.0017

 Step 2: Creating imbalance levels...

 Creating imbalance levels...
   balanced_1_1: Counter({np.int64(0): 394, np.int64(1): 394}) (ratio 1:1)
   mild_1_5: Counter({np.int64(0): 1970, np.int64(1): 394}) (ratio 5:1)
   moderate_1_10: Counter({np.int64(0): 3940, np.int64(1): 394}) (ratio 10:1)
   severe_1_50: Counter({np.int64(0): 19700, np.int64(1): 394}) (ratio 50:1)
   extreme_1_100: Counter({np.int64(0): 39400, np.int64(1): 394}) (ratio 100:1)

 Step 3: Running experiments across all ratios

 Processing BALANCED_1_1...
      Training LogisticRegression...       Training RandomForest...       Training XGBoost...  Generating explanations (5 runs)
     Processing LogisticRegression...
     Processing RandomForest...


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

     Processing XGBoost...
 extreme_1_100 completed!

 Generating Publication-Ready Plots...

📈 Creating Performance Heatmap...
   Heatmap data shape: (15, 6)
  Saved: plots/performance_heatmap.png
 Creating Stability Curves...
    Saved: plots/stability_curves.png
 Creating Model Comparison Radar...
  Saved: plots/model_radar_comparison.png
   All plots generated successfully!

 COMPLETE EXPERIMENT FINISHED WITH PLOTS!
Check folders: results/, models/, plots/

 EXPERIMENT SUCCESSFUL! Generated:
    plots/performance_heatmap.png
    plots/stability_curves.png
    plots/feature_ranking_extreme_1_100.png
    results/summary_metrics.csv
